In [14]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import pandas as pd

# Load labels
labels_path = "../../dataset/trainLabels.csv"
labels_df = pd.read_csv(labels_path)

# Display sample
print(labels_df.head())



   id       label
0   1        frog
1   2       truck
2   3       truck
3   4        deer
4   5  automobile


In [15]:
# Define parameters
image_size = (128, 128)  # Resize images
data = []
labels = []

# Folder containing all imagest
images_folder = "../../dataset/train/"  # Update this to the path where your images are stored

# Loop through the DataFrame to load images and match with labels
for index, row in labels_df.iterrows():
    img_file = os.path.join(images_folder, f"{row['id']}.png")  # Assuming images are PNGs
    if os.path.exists(img_file):
        try:
            # Load and preprocess image
            img = load_img(img_file, target_size=image_size)
            img_array = img_to_array(img) / 255.0  # Normalize to [0, 1]
            data.append(img_array)
            labels.append(row['label'])
        except Exception as e:
            print(f"Error processing {img_file}: {e}")
    else:
        print(f"Image file not found: {img_file}")

# Convert to NumPy arrays
data = np.array(data)
labels = np.array(labels)

print(f"Data shape: {data.shape}, Labels shape: {labels.shape}")


Image file not found: ../../dataset/train/2.png
Image file not found: ../../dataset/train/3.png
Image file not found: ../../dataset/train/4.png
Image file not found: ../../dataset/train/5.png
Image file not found: ../../dataset/train/6.png
Image file not found: ../../dataset/train/7.png
Image file not found: ../../dataset/train/8.png
Image file not found: ../../dataset/train/9.png
Image file not found: ../../dataset/train/11.png
Image file not found: ../../dataset/train/12.png
Image file not found: ../../dataset/train/13.png
Image file not found: ../../dataset/train/14.png
Image file not found: ../../dataset/train/15.png
Image file not found: ../../dataset/train/16.png
Image file not found: ../../dataset/train/17.png
Image file not found: ../../dataset/train/18.png
Image file not found: ../../dataset/train/19.png
Image file not found: ../../dataset/train/20.png
Image file not found: ../../dataset/train/21.png
Image file not found: ../../dataset/train/22.png
Image file not found: ../../

In [16]:
# Encode labels
encoder = LabelEncoder()
labels_encoded = encoder.fit_transform(labels)  # Convert labels to integers
labels_one_hot = to_categorical(labels_encoded)  # One-hot encode for multi-class classification

print(f"Classes: {encoder.classes_}")


Classes: ['airplane' 'automobile' 'bird' 'cat' 'deer' 'dog' 'frog' 'horse' 'ship'
 'truck']


In [17]:
X_train, X_test, y_train, y_test = train_test_split(data, labels_one_hot, test_size=0.2, random_state=42)
print(f"Training data: {X_train.shape}, Testing data: {X_test.shape}")


Training data: (536, 128, 128, 3), Testing data: (134, 128, 128, 3)


In [18]:
# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=X_train.shape[1:]),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(encoder.classes_), activation='softmax')  # Output layer
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


C:\Users\71526\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     7,372,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,393,610 (28.20 MB)

 Trainable params: 7,393,610 (28.20 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 264ms/step - accuracy: 0.0892 - loss: 2.9464 - val_accuracy: 0.1716 - val_loss: 2.2793
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 233ms/step - accuracy: 0.1349 - loss: 2.2683 - val_accuracy: 0.1791 - val_loss: 2.2156
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 234ms/step - accuracy: 0.2148 - loss: 2.2059 - val_accuracy: 0.2239 - val_loss: 2.1080
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 237ms/step - accuracy: 0.2650 - loss: 2.0718 - val_accuracy: 0.2612 - val_loss: 2.0340
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 230ms/step - accuracy: 0.3228 - loss: 1.9268 - val_accuracy: 0.2761 - val_loss: 1.9432
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 227ms/step - accuracy: 0.3937 - loss: 1.7658 - val_accuracy: 0.3284 - val_loss: 1.8551
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 229ms/step - accuracy: 0.4778 - loss: 1.5362 - val_accuracy: 0.2612 - val_loss: 1.9141
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 238ms/step - accuracy: 0.5096 - loss: 1.4110 - val_accuracy: 0.3284 - val_loss:

In [20]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.3142 - loss: 1.9991
Test Loss: 2.023649215698242
Test Accuracy: 0.305970162153244
